### Cuestionario
Para guiar nuestro trabajo, nuestro Team Líder nos facilita un cuestionario. Entonces, usando los datos ingestados anteriormente debemos responder:

1 - ¿Cuáles fueron los 5 Estados con mayor ocupación hospitalaria por COVID? Criterio de ocupación por cama común. Considere la cantidad de camas ocupadas con pacientes confirmados y tome como referencia los 6 primeros meses del 2020 - recuerde incluir la cifra de infectados en esos meses (acumulativo). ¿Influye el rango etario en este comportamiento?

2 - Analice la ocupación de camas (Común) por COVID en el Estado de Nueva York durante la cuarentena establecida e indique:

Intervalos de crecimiento y decrecimiento
Puntos críticos (mínimos y máximos)
3 - ¿Cuáles fueron los cinco Estados que más camas UCI -Unidades de Cuidados Intensivos- utilizaron durante el año 2020? La medición debe realizarse en términos absolutos.

4 - ¿Qué cantidad de camas se utilizaron, por Estado, para pacientes pediátricos con COVID durante el 2020?

5 - ¿Qué porcentaje de camas UCI corresponden a casos confirmados de COVID-19? Agrupe por Estado.

6 - ¿Cuántas muertes por covid hubo, por Estado, durante el año 2021?

7 - ¿Qué relación presenta la falta de personal médico, con la cantidad de muertes por covid durante el año 2021?

8 - Siguiendo las respuestas anteriores, ¿cuál fue el peor mes de la pandemia para USA en su conjunto? Puede utilizar otras medidas que considere necesarias.

9 - ¿Qué recomendaciones haría, ex post, con respecto a los recursos hospitalarios y su uso?



In [5]:
import pandas as pd
import streamlit as st
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [6]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("healthdata.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(healthdata.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("g62h-syeh", limit=50000)

# Convert to pandas DataFrame
resultados_df = pd.DataFrame.from_records(results)

2022-08-11 10:23:00.711 WARNING root: Requests made without an app_token will be subject to strict throttling limits.


In [7]:
resultados_df.shape

(47909, 134)

In [8]:
resultados_df.to_csv('./dataset/data_covid.csv')

In [9]:
df = pd.read_csv('./dataset/data_covid.csv')

In [10]:
# Convertimos fechas en datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [11]:
# Seleccionamos columnas que vemos mas relevantes para el informe
selecc_col_dic = {   'state' : 'Estado', # Estado 
                 'date' : 'Fecha',  # Fecha
                 'critical_staffing_shortage_today_yes': 'Reporta_escasez_de_personal', # Hospitales que informan escasez de personal
                 'critical_staffing_shortage_today_no': 'Reportan_sin_escasez_de_personal', # Hospitales
                 'critical_staffing_shortage_today_not_reported': 'No_reportan_sobre_escasez_de_personal', #Hospitales
                 'inpatient_beds': 'Camas_hospital', #hospital_camas  
                 'inpatient_beds_used_covid': 'Camas_usadas_sosp_conf', # hospital_camas_usadas_sospech_confir_covid
                 'previous_day_admission_adult_covid_confirmed': 'Admision_adulto_conf_dia_previo', # día_anterior_admisión_adulto_covid_confirmado
                 'previous_day_admission_pediatric_covid_confirmed': 'Admision_pediatrico_conf_dia_previo', # día_anterior_ingreso_pediátrico_covid_confirmado
                  'staffed_adult_icu_bed_occupancy': 'Camas_UCI_ocupadas_general', # numero_total_camas_UCI_ocupadas_general
                  'staffed_icu_adult_patients_confirmed_covid': 'Camas_UCI_adultos_ocupadas_conf', # numero_camas_UCI_ocupadas_Covid_confirmado_adultos
                  'total_adult_patients_hospitalized_confirmed_covid': 'Camas_comunes_adulto_ocupadas_conf', # numero_camas_comunes_ocupadas_Covid_confirmado
                  'total_pediatric_patients_hospitalized_confirmed_covid': 'Camas_comunes_pediatrico_ocupadas_conf', # numero_camas_comunes_ocupadas_Covid_confirmado_pediatrico
                  'total_staffed_adult_icu_beds': 'Camas_UCI_adultos', # numero_camas_UCI_adultos
                  'inpatient_beds_utilization': 'Porcentaje_utilizacion_camas', # porcentaje de camas utilizadas en el estado
                  #'geocoded_state', # estado_geocodificado 
                  'previous_day_admission_adult_covid_confirmed_18_19': 'Rango_18_19_confirmados_hospitalizados', # rango etario
                  'previous_day_admission_adult_covid_confirmed_20_29': 'Rango_20_29_confirmados_hospitalizados', # rango etario
                  'previous_day_admission_adult_covid_confirmed_30_39': 'Rango_30_39_confirmados_hospitalizados', # rango etario
                  'previous_day_admission_adult_covid_confirmed_40_49': 'Rango_40_49_confirmados_hospitalizados', # rango etario
                  'previous_day_admission_adult_covid_confirmed_50_59': 'Rango_50_59_confirmados_hospitalizados', # rango etario
                  'previous_day_admission_adult_covid_confirmed_60_69': 'Rango_60_69_confirmados_hospitalizados',  # rango etario
                  'previous_day_admission_adult_covid_confirmed_70_79': 'Rango_70_79_confirmados_hospitalizados', # rango etario
                  'previous_day_admission_adult_covid_confirmed_80': 'Rango_80_mas_confirmados_hospitalizados',     # rango etario
                  'previous_day_admission_adult_covid_confirmed_unknown': 'Rango_desconocido_confirmados_hospitalizados', # rango etario
                  'deaths_covid': 'Muertes_covid', # muertes covid conf y sospechosos
                   'all_pediatric_inpatient_beds': 'Camas_pediatricas', # Total camas pediatricas
                   'previous_day_admission_pediatric_covid_confirmed_0_4': 'Rango_0_4_confirmados_hospitalizados',
                   'previous_day_admission_pediatric_covid_confirmed_5_11': 'Rango_5_11_confirmados_hospitalizados',
                   'previous_day_admission_pediatric_covid_confirmed_12_17': 'Rango_12_17_confirmados_hospitalizados'
                }

In [12]:
df = df[list(selecc_col_dic.keys())]
df.rename(selecc_col_dic, axis= 1, inplace= True)


In [13]:
fecha_max = str(df.Fecha.values.max())[:10]
fecha_min = str(df.Fecha.values.min())[:10]

##### 1 - ¿Cuáles fueron los 5 Estados con mayor ocupación hospitalaria por COVID? Criterio de ocupación por cama común. Considere la cantidad de camas ocupadas con pacientes confirmados y tome como referencia los 6 primeros meses del 2020 - recuerde incluir la cifra de infectados en esos meses (acumulativo). ¿Influye el rango etario en este comportamiento?


In [14]:
# Filtramos dataset
df_fechas = df[(df['Fecha'] >= fecha_min) & (df['Fecha'] <= '2020-06-30')]
df_agrup_state = df_fechas[['Estado', 'Camas_comunes_adulto_ocupadas_conf', 'Camas_comunes_pediatrico_ocupadas_conf']].groupby('Estado').sum().sort_values(by = 'Camas_comunes_adulto_ocupadas_conf' ,ascending= False).head(5)

In [15]:
#Grafico
figura1 = px.bar(df_agrup_state, 
                    title= 'Top 5 Estados con mayor camas (comunes utilizadas)',
                    )
figura1.show()

##### 2 - Analice la ocupación de camas (Común) por COVID en el Estado de Nueva York durante la cuarentena establecida e indique:

Intervalos de crecimiento y decrecimiento
Puntos críticos (mínimos y máximos)

In [16]:
df_fechas_NY = df[df['Fecha'] <= '2022-08-01'] 
df_fechas_NY = df_fechas_NY[df_fechas_NY['Estado'] == 'NY'][['Fecha', 'Camas_comunes_adulto_ocupadas_conf']].sort_values(by= 'Fecha')
df_fechas_NY.dropna(inplace= True)

In [17]:
# Grafico
figura2 = px.line(df_fechas_NY, x= 'Fecha', y= 'Camas_comunes_adulto_ocupadas_conf')
figura2.show()

In [18]:
max_2020 = df_fechas_NY.Camas_comunes_adulto_ocupadas_conf.max()
fecha_punto_max = str(df_fechas_NY[(df_fechas_NY['Camas_comunes_adulto_ocupadas_conf'] == max_2020)].Fecha.values)[2:12]
print('Punto Max:', max_2020, 'Fecha:', fecha_punto_max)

Punto Max: 12391.0 Fecha: 2022-01-11


##### 3 - ¿Cuáles fueron los cinco Estados que más camas UCI -Unidades de Cuidados Intensivos- utilizaron durante el año 2020? La medición debe realizarse en términos absolutos.


In [19]:
df_2020 = df[(df['Fecha'] <= '2020-12-31')]
df_agrup_state_UCI = df_2020[['Estado', 'Camas_UCI_adultos_ocupadas_conf']].groupby('Estado').sum()
df_agrup_state_UCI = df_agrup_state_UCI.sort_values(by = 'Camas_UCI_adultos_ocupadas_conf' ,ascending= False).head(5)

In [20]:
fig_UCI = px.bar(df_agrup_state_UCI)
fig_UCI.show()

##### 4 - ¿Qué cantidad de camas se utilizaron, por Estado, para pacientes pediátricos con COVID durante el 2020?


In [21]:
df_pediatrico = df_2020[['Estado',
         'Camas_comunes_pediatrico_ocupadas_conf']]
df_pediatrico = df_pediatrico.groupby(['Estado']).sum().sort_values(by = 'Camas_comunes_pediatrico_ocupadas_conf' ,ascending= False).head(5)

In [22]:
fig_pediatrico = px.bar(df_pediatrico)
fig_pediatrico.show()

##### 5 - ¿Qué porcentaje de camas UCI corresponden a casos confirmados de COVID-19? Agrupe por Estado.


In [23]:
df_UCI = df[['Estado', 'Camas_UCI_adultos_ocupadas_conf', 'Camas_UCI_ocupadas_general', 'Camas_UCI_adultos' ]]
df_UCI.dropna(inplace= True)
df_UCI['Porc_Covid_ocupadas'] = (df_UCI['Camas_UCI_adultos_ocupadas_conf']/(df_UCI['Camas_UCI_adultos_ocupadas_conf']+ df_UCI['Camas_UCI_ocupadas_general']))*100
df_UCI['Porc_Covid_general'] = (df_UCI['Camas_UCI_adultos_ocupadas_conf']/df_UCI['Camas_UCI_adultos'])*100

C:\Users\Mauricio\AppData\Local\Temp\ipykernel_18308\3198669486.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Mauricio\AppData\Local\Temp\ipykernel_18308\3198669486.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Mauricio\AppData\Local\Temp\ipykernel_18308\3198669486.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [24]:
df_UCI.head()

,Estado,Camas_UCI_adultos_ocupadas_conf,Camas_UCI_ocupadas_general,Camas_UCI_adultos,Porc_Covid_ocupadas,Porc_Covid_general
0,ID,97.0,232.0,310.0,29.483283,31.290323
1,VT,0.0,75.0,110.0,0.000000,0.000000
2,AR,217.0,776.0,996.0,21.852971,21.787149
3,VI,1.0,10.0,20.0,9.090909,5.000000
4,VI,1.0,11.0,20.0,8.333333,5.000000


In [25]:
df_UCI[(df_UCI['Estado'] == 'AL') & (df_UCI['Camas_UCI_adultos'] > 0)]['Camas_UCI_adultos'].min()

54.0

In [26]:
df_UCI = df_UCI.groupby(by= 'Estado').max()

In [27]:
df_UCI.head(10)

,Camas_UCI_adultos_ocupadas_conf,Camas_UCI_ocupadas_general,Camas_UCI_adultos,Porc_Covid_ocupadas,Porc_Covid_general
Estado,,,,,
AK,53.0,136.0,162.0,33.125000,40.769231
AL,906.0,1678.0,1725.0,100.000000,inf
AR,500.0,933.0,1127.0,55.213270,108.372093
AS,2.0,7.0,7.0,25.000000,28.571429
AZ,1085.0,1848.0,2443.0,51.671975,105.990017
CA,4926.0,8387.0,9434.0,46.591422,66.281310
CO,533.0,1168.0,1634.0,34.101088,40.470767
CT,297.0,721.0,1136.0,100.000000,inf
DC,116.0,311.0,404.0,100.000000,inf


In [28]:
df_UCI.reset_index(inplace= True)

In [29]:
figura5 = px.bar(df_UCI, x= 'Estado', y= 'Porc_Covid_ocupadas')
figura5.show()

In [30]:
figura5 = px.bar(df_UCI, x= 'Estado', y= 'Porc_Covid_general')
figura5.show()

##### 6 - ¿Cuántas muertes por covid hubo, por Estado, durante el año 2021?

In [45]:
df_muertes_covid_2021 = df[(df['Fecha'] >= '2021-01-01') & (df['Fecha'] <= '2021-12-31')][['Estado', 'Muertes_covid']]
df_muertes_covid_2021 = df_muertes_covid_2021.groupby(by= 'Estado').sum()
df_muertes_covid_2021 = df_muertes_covid_2021.sort_values(by= 'Muertes_covid', ascending= False).head(20)

In [46]:
figura6 = px.bar(df_muertes_covid_2021)
figura6.show()

##### 7 - ¿Qué relación presenta la falta de personal médico, con la cantidad de muertes por covid durante el año 2021?


In [33]:
df_esca_muertes = df[['Fecha', 'Reporta_escasez_de_personal', 'Reportan_sin_escasez_de_personal', 'No_reportan_sobre_escasez_de_personal', 'Muertes_covid']].sort_values(by= 'Fecha')

In [34]:
df_esca_muertes = df_esca_muertes[['Fecha', 'Reporta_escasez_de_personal', 'Muertes_covid']]

In [35]:
df_esca_muertes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47909 entries, 4501 to 45400
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Fecha                        47909 non-null  datetime64[ns]
 1   Reporta_escasez_de_personal  47909 non-null  int64         
 2   Muertes_covid                47220 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.5 MB


In [36]:
df_esca_muertes.astype({'Reporta_escasez_de_personal': 'float64'})

,Fecha,Reporta_escasez_de_personal,Muertes_covid
4501,2020-01-01,0.0,0.0
17352,2020-01-01,0.0,0.0
14043,2020-01-01,0.0,0.0
16481,2020-01-01,0.0,0.0
13718,2020-01-01,0.0,0.0
...,...,...,...
39839,2022-08-10,3.0,1.0
41749,2022-08-10,1.0,0.0
45545,2022-08-10,2.0,3.0
34079,2022-08-10,2.0,1.0


In [37]:
df_esca_W = df_esca_muertes[['Fecha', 'Reporta_escasez_de_personal']].resample('W', on= 'Fecha').mean()
df_muertes_W = df_esca_muertes[['Fecha','Muertes_covid']].resample('W', on= 'Fecha').sum()
df_esca_W.reset_index(inplace= True)
df_muertes_W.reset_index(inplace= True)

In [38]:
x= df_esca_W.Fecha.values
y1 = df_esca_W.Reporta_escasez_de_personal.values
y2 = df_muertes_W.Muertes_covid.values

In [39]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

heat = go.Heatmap(
    z=df_esca_muertes.corr(),
    x=df_esca_muertes.corr().columns,
    y=df_esca_muertes.corr().columns,
    zmin = - 0.25, # Sets the lower bound of the color domain
    zmax = 1,
    xgap = 1, # Sets the horizontal gap (in pixels) between bricks
    ygap = 1,
    colorscale = 'RdBu'
)

title = 'Matriz de Correlacion'

layout = go.Layout(
    title_text=title, 
    title_x=0.5, 
    width=400, 
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed'
)

fig=go.Figure(data=[heat], layout=layout)
fig.show()

In [40]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Muertes por Covid", "Hospitales que Reportan Escasez de Personal"))

fig.append_trace(go.Scatter(
    x= x,
    y= y2,
), row=1, col=1)

fig.append_trace(go.Scatter(
    x= x,
    y=y1,
), row=2, col=1)

fig.update_layout(height=600, width=600)
fig.show()

##### 8 - Siguiendo las respuestas anteriores, ¿cuál fue el peor mes de la pandemia para USA en su conjunto? Puede utilizar otras medidas que considere necesarias.

In [41]:
df_agrup_fecha = df[['Fecha', 'Camas_comunes_adulto_ocupadas_conf', 'Camas_comunes_pediatrico_ocupadas_conf', 'Camas_UCI_adultos_ocupadas_conf','Reporta_escasez_de_personal', 'Muertes_covid']]
df_agrup_fecha['Hospitalizados'] = df_agrup_fecha['Camas_comunes_adulto_ocupadas_conf'] + df_agrup_fecha['Camas_comunes_pediatrico_ocupadas_conf']

C:\Users\Mauricio\AppData\Local\Temp\ipykernel_18308\126112518.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
df_agrup_fecha = df_agrup_fecha.groupby('Fecha', as_index= False).sum()
df_agrup_fecha.drop(['Camas_comunes_pediatrico_ocupadas_conf','Camas_comunes_adulto_ocupadas_conf'], axis=1, inplace= True)
df_agrup_fecha.rename({'Camas_UCI_adultos_ocupadas_conf':'Hospitalizados_UCI'}, axis= 1, inplace= True)

In [47]:
#df_agrup_fecha['Total_Hospitalizados'] = df_agrup_fecha['Hospitalizados_UCI']+

In [48]:
df_agrup_fecha.head()

,Fecha,Hospitalizados_UCI,Reporta_escasez_de_personal,Muertes_covid,Hospitalizados
0,2020-01-01,0.0,0,0.0,0.0
1,2020-01-02,397.0,16,29.0,1801.0
2,2020-01-03,397.0,16,29.0,1801.0
3,2020-01-04,397.0,16,29.0,1801.0
4,2020-01-05,397.0,16,29.0,1801.0


In [49]:
fig = make_subplots(rows=4, cols=1,
                    subplot_titles=("Hospitalizados", 'Hospitalizados_UCI','Muertes por Covid', "Hospitales que Reportan Escasez de Personal"))

fig.append_trace(go.Scatter(
    x= df_agrup_fecha.Fecha,
    y= df_agrup_fecha.Hospitalizados,
), row=1, col=1)

fig.append_trace(go.Scatter(
    x= df_agrup_fecha.Fecha,
    y= df_agrup_fecha.Hospitalizados_UCI,
), row=2, col=1)

fig.append_trace(go.Scatter(
    x= df_agrup_fecha.Fecha,
    y=df_agrup_fecha.Muertes_covid,
), row=3, col=1)

fig.append_trace(go.Scatter(
    x= df_agrup_fecha.Fecha,
    y=df_agrup_fecha.Reporta_escasez_de_personal,
), row=4, col=1)

fig.update_layout(height=600, width=600)
fig.show()


In [50]:

figura8 = px.line(df_agrup_fecha,
                    x= df_agrup_fecha.Fecha,
                    y= [df_agrup_fecha.Hospitalizados, df_agrup_fecha.Hospitalizados_UCI]#, df_agrup_fecha.Muertes_covid, df_agrup_fecha.Reporta_escasez_de_personal]
                    )
                    
figura8.show()

In [51]:
df.columns

Index(['Estado', 'Fecha', 'Reporta_escasez_de_personal',
       'Reportan_sin_escasez_de_personal',
       'No_reportan_sobre_escasez_de_personal', 'Camas_hospital',
       'Camas_usadas_sosp_conf', 'Admision_adulto_conf_dia_previo',
       'Admision_pediatrico_conf_dia_previo', 'Camas_UCI_ocupadas_general',
       'Camas_UCI_adultos_ocupadas_conf', 'Camas_comunes_adulto_ocupadas_conf',
       'Camas_comunes_pediatrico_ocupadas_conf', 'Camas_UCI_adultos',
       'Porcentaje_utilizacion_camas',
       'Rango_18_19_confirmados_hospitalizados',
       'Rango_20_29_confirmados_hospitalizados',
       'Rango_30_39_confirmados_hospitalizados',
       'Rango_40_49_confirmados_hospitalizados',
       'Rango_50_59_confirmados_hospitalizados',
       'Rango_60_69_confirmados_hospitalizados',
       'Rango_70_79_confirmados_hospitalizados',
       'Rango_80_mas_confirmados_hospitalizados',
       'Rango_desconocido_confirmados_hospitalizados', 'Muertes_covid',
       'Camas_pediatricas', 'Rango_0

In [52]:
df_etario = df[['Fecha', 'Rango_0_4_confirmados_hospitalizados',
              'Rango_5_11_confirmados_hospitalizados',
              'Rango_12_17_confirmados_hospitalizados',
              'Rango_18_19_confirmados_hospitalizados',
              'Rango_20_29_confirmados_hospitalizados',
              'Rango_30_39_confirmados_hospitalizados',
              'Rango_40_49_confirmados_hospitalizados',
              'Rango_50_59_confirmados_hospitalizados',
              'Rango_60_69_confirmados_hospitalizados',
              'Rango_70_79_confirmados_hospitalizados',
              'Rango_80_mas_confirmados_hospitalizados']]

In [53]:
dicc_rename_rango = {'Rango_0_4_confirmados_hospitalizados': '0_4',
                'Rango_5_11_confirmados_hospitalizados': '5_11',
                'Rango_12_17_confirmados_hospitalizados': '12_17',
                'Rango_18_19_confirmados_hospitalizados': '18_19',
                'Rango_20_29_confirmados_hospitalizados': '20-29',
                'Rango_30_39_confirmados_hospitalizados': '30_39',
                'Rango_40_49_confirmados_hospitalizados': '40-49',
                'Rango_50_59_confirmados_hospitalizados': '50_59',
                'Rango_60_69_confirmados_hospitalizados':  '60_69',
                'Rango_70_79_confirmados_hospitalizados':  '70_79',
                'Rango_80_mas_confirmados_hospitalizados':  '80+'}

In [54]:
df_etario.rename(dicc_rename_rango, axis= 1, inplace= True)

C:\Users\Mauricio\AppData\Local\Temp\ipykernel_18308\2954197031.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
dicc_etario= {}
for c in df_etario.drop('Fecha', axis=1).columns:
    dicc_etario[c] = df_etario[c].sum()


In [56]:
df_etario = pd.DataFrame([[key, dicc_etario[key]] for key in dicc_etario.keys()], columns=['Rango', 'Hopitalizados_Confirm'])

In [57]:
figura9 = px.bar(df_etario, 
                 x= df_etario.Rango,
                 y= df_etario.Hopitalizados_Confirm,
                 title= 'Hospitalizados por Rango Etario')
figura9.show()

##### 9 - ¿Qué recomendaciones haría, ex post, con respecto a los recursos hospitalarios y su uso?